# Реализация модели двустороннего защищенного чата в мессенджере по спецификации Signal

In [43]:
IS_DEBUG = 1

In [44]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [46]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# КГПСЧ
from Crypto.Random import get_random_bytes, random

# HKDF
from Crypto.Protocol.KDF import HKDF

# Вспомогательные радости
from collections import namedtuple
import dataclasses
import binascii
import pickle

----

In [47]:
def as_hex(blob: bytes):
    """
    Перевод блоба в шестнадцатиричное представление
    """
    
    return binascii.hexlify(blob).decode()

----
## Реализация оберток над криптографическими алгоритмами и примитивами

In [48]:
#
# Описание сути следует смотреть здесь:
# https://signal.org/docs/specifications/x3dh/#publishing-keys
#

KeyBundle = namedtuple('KeyBundle', ['IK',    # Ключ-идентификатор (Identity key)
                                     'SPK',   # Подписанный предключ (Signed prekey)
                                     'Sig',   # Подпись предключа (Prekey signature)
                                     'OPKs',  # Коллекция одноразовых предключей (One-time prekeys)
                                    ])

In [49]:
KeyPair = namedtuple('KeyPair', ['private', 'public'])

In [50]:
Message = namedtuple('Message', ['public_key',  # Открытый ключ DH отправляющего
                                 'id',          # Идентитфикатор сообщения
                                 'data',        # Данные сообщения
                                ])

----

In [51]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes, associated_data: bytes = b''):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = gost3412.GOST3412Kuznechik(self._key)
        cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, associated_data)
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes, associated_data: bytes = b''):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        try:
            encrypter = gost3412.GOST3412Kuznechik(self._key)
            cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())

            ct_with_tag = ciphertext + tag

            return cipher.open(nonce, ct_with_tag, associated_data)
        except ValueError as e:
            return None
        
        
    @staticmethod
    def _block_size():
        return gost3412.GOST3412Kuznechik.blocksize
    
    
    @staticmethod
    def _generate_nonce():
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

---

In [52]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [53]:
def hash_digest(hasher_type, *args):
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [54]:
class Curve(object):
    """
    Обертка над эллиптической кривой. Для простоты это одна из кривых,
    определенных в ГОСТ 34.10
    """
    
    CURVE = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        
        
    @staticmethod
    def generate_private_key():
        """
        Генерирование закрытого ключа
        """
        
        return gost3410.prv_unmarshal(get_random_bytes(64))
    
    
    @staticmethod
    def get_public_key(private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return gost3410.public_key(Curve.CURVE, private_key)
        
        
    @staticmethod
    def multiply_by_scalar(scalar, point):
        """
        Умножение точки эллиптической кривой на скаляр 
        """

        return Curve.CURVE.exp(scalar, *point)

    
    @staticmethod
    def add_points(lhs, rhs):
        """
        Сложение двух точек эллиптической кривой
        """

        return Curve.CURVE._add(*lhs, *rhs)

----
## Элементы протоколов из спецификации Signal

In [55]:
class KdfChainInternal(object):
    """
    Вспомогательный класс, описывающий KDF-цепочку
    """
    
    def __init__(self, key, key_length):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        self._key_length = key_length
        self._kdf_key = key
    
    
    def reset(self, key):
        """
        Сброс состояния, запись нового ключа в качестве
        первоначального
        """
        
        self._kdf_key = key
        
        
    def _kdf_update(self, data):
        self._kdf_key, message_key = HKDF(self._kdf_key, self._key_length, 
                                          data, Streebog, 2)
        return message_key

In [56]:
class KdfChain(KdfChainInternal):
    """
    Класс, описывающий KDF-цепочку
    Новые ключи вырабатываются на основе выработанного 
    секрета Диффи-Хеллмана
    https://signal.org/docs/specifications/doubleratchet/#kdf-chains
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(KdfChain, self).__init__(key, key_length)
        
        
    def step(self, serialized_dh_key):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(serialized_dh_key)

In [57]:
class SymmetricRatchet(KdfChainInternal):
    """
    Симметричный храповик, который генерирует
    новые ключи только на основе своего состояния
    https://signal.org/docs/specifications/doubleratchet/#symmetric-key-ratchet
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(SymmetricRatchet, self).__init__(key, key_length)
        
        
    def step(self):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(b'\xFF' * 32)

-----

In [58]:
class DiffieHellmanRatchet(object):
    """
    DH-храповик
    https://signal.org/docs/specifications/doubleratchet/#diffie-hellman-ratchet
    """
    
    def __init__(self):
        """
        Инициализация, выработка начальной ключевой пары
        """
        
        self._current_foreign_public_key = None
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
    
    def is_uninitialized(self):
        """
        Проверка того, что открытый ключ другой стороны отсутствует
        """
        
        return self._current_foreign_public_key is None
    
    
    def initialize(self, public_key):
        """
        Инициализация открытого ключа другой стороны
        """
        
        self._current_foreign_public_key = public_key
        
        
    def is_same_public_key(self, public_key):
        """
        Проверка равенства открытых ключей
        """
        
        if self._current_foreign_public_key is None:
            return False
        
        return public_key == self._current_foreign_public_key
    
    
    def step(self, public_key=None):
        """
        Выработка общего секрета Диффи-Хеллмана
        """
        
        if self._current_key_pair is None:
            raise RuntimeError('Uninitialized DH Ratchet used to produce a shared secret')
        
        if public_key is not None:
            self._current_foreign_public_key = public_key
            
        return Curve.multiply_by_scalar(self._current_key_pair.private, self._current_foreign_public_key)
    
    
    def generate_new_key_pair(self):
        """
        Генерирование новой ключевой пары
        """
        
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._current_key_pair.public
    
    
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return KeyPair(private_key, public_key)

----
## Участники протоколов из спецификации Signal

In [59]:
class SignalServer(object):
    """
    Класс сервера Signal
    """
    
    def __init__(self):
        """
        Инициализация
        """
        
        #
        # Коллекция ключевых данных клиентов, индексируемая по номеру телефона
        #
        
        self._key_bundles = {}
        
        #
        # Коллекция очередей сообщения, индексируемая по паре номеров, 
        # представляющих направление передачи
        #
        
        self._messages = {}
        
        
    def register(self, phone_number: str, keys: KeyBundle):
        """
        Регистрация пользователя по номеру телефона. Сохраняет набор ключей
        """
        
        if phone_number in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is already registered')
            
        trace('[SignalServer]', f'Registered client with phone number {phone_number}')
            
        self._key_bundles[phone_number] = keys
        
        
    def get_public_keys(self, phone_number: str):
        """
        Получение ключей пользователя по его номеру телефона
        """
        
        if phone_number not in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is not registered yet')
            
        #
        # Достану ключи
        #
        
        keys = self._key_bundles[phone_number]
        one_time_keys = keys.OPKs
        
        #
        # Возьму случайный из одноразовых ключей
        #
        
        idx = random.randint(0, len(one_time_keys) - 1)
        result = KeyBundle(keys.IK, keys.SPK, keys.Sig, (one_time_keys.pop(idx), idx))
        
        trace('[SignalServer]', f'Using OPK at {idx}')
        
        #
        # Сохраню обновленный пакет ключей (без одного одноразового)
        #
        
        self._key_bundles[phone_number] = KeyBundle(keys.IK, keys.SPK, keys.Sig, one_time_keys)
        
        return result
    
    
    def push_message(self, phone_from: str, phone_to: str, message: Message):
        """
        Помещение сообщения в очередь
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            self._messages[search_key] = []
            
        trace('[SignalServer]', f'Saved message with id {message.id} from {phone_from} to {phone_to}')
            
        self._messages[search_key].append(message)
        
        
    def peek_message(self, phone_from, phone_to):
        """
        Получение сообщения из очереди
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            return None
        
        if self._messages[search_key] == []:
            return None
        
        return self._messages[search_key].pop(0)

In [60]:
class SignalClient(object):
    """
    Класс клиента Signal
    """
    
    X3DH_MSG_ID      = 'X3DH_MSG'
    DH_RATCHET_PK_ID = 'DH_RATCHET_PK'
    
    
    @dataclasses.dataclass
    class SessionState:
        received: bool
        dh_ratchet: DiffieHellmanRatchet
        root_kdf_chain: KdfChain
        send_ratchet: SymmetricRatchet
        recv_ratchet: SymmetricRatchet
        msg_id: int
    
    
    def __init__(self, phone_number, one_time_keys=10, *, fake_ik=False):
        """
        Инициализация, генерирование ключей
        """
        
        self._phone_number  = phone_number
        
        self._identity_key  = SignalClient._generate_key_pair()
        self._signed_prekey = SignalClient._generate_key_pair()
        self._ot_prekeys    = [SignalClient._generate_key_pair() for _ in range(one_time_keys)]
        
        digest = hash_digest(Streebog, pickle.dumps(self._signed_prekey.public))
        self._signature = gost3410.sign(Curve.CURVE, self._identity_key.private, digest)
        
        if fake_ik:
            #
            # Демонстрация неудачного выполнения, заменю идентификационный ключ
            #
            
            self._identity_key  = SignalClient._generate_key_pair()
        
        self._sessions = {}
        
        trace('[SignalClient]', f'Identity key: {self._identity_key}')
        trace('[SignalClient]', f'Signed prekey: {self._signed_prekey}')
        trace('[SignalClient]', f'Signature: {as_hex(self._signature)}')
        trace('[SignalClient]', f'One time keys: {len(self._ot_prekeys)}')
        
        
    def send_message(self, phone_number: str, message: str, server: SignalServer):
        """
        Отправка сообщения, установление сессии, если нужно
        """
        
        if phone_number not in self._sessions:
            #
            # Сессии еще нет, поэтому установлю соединение
            #
            
            return self._x3dh_init_session(phone_number, message, server)
        
        #
        # Так как протокол X3DH асинхронный, то я прямо сразу и отправлю сообщение
        #
        
        if self._sessions[phone_number].received:
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            self._sessions[phone_number].received = False
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key   = self._sessions[phone_number].dh_ratchet.step()
            
            trace('[SignalClient]', 'Root KDF chain step')
            send_key = self._sessions[phone_number].root_kdf_chain.step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Sending ratchet reset')
            self._sessions[phone_number].send_ratchet.reset(send_key)
            
        #
        # Вырабатывается ключ с помощью симметричного храповика для отправки
        # На ключе шифруется само сообщение
        #
        
        trace('[SignalClient]', 'Sending ratchet step')
        message_key = self._sessions[phone_number].send_ratchet.step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as encryption key')
        encrypted_message = KuznyechikMGM(message_key).encrypt(message.encode())
        
        message_id = self._sessions[phone_number].msg_id
        public_key = self._sessions[phone_number].dh_ratchet.public_key
        
        self._sessions[phone_number].msg_id += 1
        
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     message_id, 
                                                                     encrypted_message))
        
        
    def receive_message(self, phone_number: str, server: SignalServer):
        """
        Прием сообщения, установление сессии, если надо
        """
        
        message = server.peek_message(phone_number, self.phone_number)
        if message is None:
            trace('[SignalClient]', f'No message for {self.phone_number} from {phone_number}')
            return
        
        public_key, message_id, data = message
        
        trace('[SignalClient]', f'Received message with id {message_id} from {phone_number}')
        
        if message_id == SignalClient.X3DH_MSG_ID:
            #
            # Это инициирующее сообщения для установки сессии
            #
            
            return self._x3dh_accept_session(phone_number, data, server)
        
        if message_id == SignalClient.DH_RATCHET_PK_ID:
            #
            # Это нам ключ открытый прислали для инициализации
            #
            
            if not self._sessions[phone_number].dh_ratchet.is_uninitialized():
                raise RuntimeError(f'Cannot initialize DH ratchet for {self.phone_number} twice')
                
            return self._sessions[phone_number].dh_ratchet.initialize(public_key)
        
        #
        # Во-первых, факт приема зафиксирую 
        #
        
        self._sessions[phone_number].received = True
        
        #
        # Не сменился ли открытый ключ собеседника?
        #
        
        if not self._sessions[phone_number].dh_ratchet.is_same_public_key(public_key):
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key = self._sessions[phone_number].dh_ratchet.step(public_key)
            self._sessions[phone_number].dh_ratchet.generate_new_key_pair()
            
            trace('[SignalClient]', 'Root KDF chain step')
            recv_key = self._sessions[phone_number].root_kdf_chain.step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Receiving ratchet reset')
            self._sessions[phone_number].recv_ratchet.reset(recv_key)
            
        #
        # Делаю шаг храповика приема
        #
            
        trace('[SignalClient]', 'Receiving ratchet step')
        message_key = self._sessions[phone_number].recv_ratchet.step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as decryption key')
        decrypted_message = KuznyechikMGM(message_key).decrypt(*data)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
            
        return decrypted_message.decode()
        
        
    @property
    def phone_number(self):
        """
        Получение номер телефона
        """
        
        return self._phone_number
    
    
    @property
    def public_keys(self):
        """
        Получение открытых ключей
        """
        
        return KeyBundle(self._identity_key.public, self._signed_prekey.public, 
                         self._signature, [prekey.public for prekey in self._ot_prekeys])
        
        
    def _x3dh_init_session(self, phone_number: str, message: str, server: SignalServer):
        IKb, SPKb, Sigb, (OPKb, idx) = server.get_public_keys(phone_number)
        
        #
        # Проверяю подпись
        #
        
        digest = hash_digest(Streebog, pickle.dumps(SPKb))
        if not gost3410.verify(Curve.CURVE, IKb, digest, Sigb):
            trace('[SignalClient]', f'Cannot verify signature of client with phone {phone_number}')
            return
        
        #
        # Эфемерный ключ
        #
        
        EKa = SignalClient._generate_key_pair()
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._identity_key, SPKb)
        DH2 = SignalClient._dh(EKa, IKb)
        DH3 = SignalClient._dh(EKa, SPKb)
        DH4 = SignalClient._dh(EKa, OPKb)
        
        #
        # Вырабатываю ключ и создаю сообщение
        # В качестве данных я отправлю шифртекст сообщения,
        # идентификационный ключ, эфемерный ключ и индекс
        # использованного одноразового ключа
        #
        # Протокол разрешает использовать выработанный ключ прямо
        # для зашифрования первого сообщения: An initial ciphertext 
        # encrypted with some AEAD encryption scheme [4] using AD 
        # as associated data and using an encryption key which is 
        # either SK or the output from some cryptographic PRF keyed by SK.
        #
        
        AD = pickle.dumps(self._identity_key.public) + pickle.dumps(IKb)
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        #
        # А теперь требуется создать состояние
        #
        
        self._initialize_state(phone_number, root_key)
        
        encrypted_message = KuznyechikMGM(root_key).encrypt(message.encode(), AD)
        message = (encrypted_message, self._identity_key.public, EKa.public, idx)
        
        server.push_message(self.phone_number, phone_number, 
                            Message(None, SignalClient.X3DH_MSG_ID, message))
        
    
    def _x3dh_accept_session(self, phone_number: str, x3dh_parameters: Message, server: SignalServer):
        encrypred_message, IKa, EKa, idx = x3dh_parameters
        
        #
        # Получу нужный одноразовый ключ
        #
        
        OPKb = self._ot_prekeys.pop(idx)
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._signed_prekey, IKa)
        DH2 = SignalClient._dh(self._identity_key, EKa)
        DH3 = SignalClient._dh(self._signed_prekey, EKa)
        DH4 = SignalClient._dh(OPKb, EKa)
        
        #
        # Вырабатываю ключ и читаю сообщение
        #
        
        AD = pickle.dumps(IKa) + pickle.dumps(self._identity_key.public)
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        decrypted_message = KuznyechikMGM(root_key).decrypt(*encrypred_message, AD)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
        
        #
        # Теперь нужно инициализировать состояние и отослать открытый ключ
        #
        
        self._initialize_state(phone_number, root_key)
        
        public_key = self._sessions[phone_number].dh_ratchet.public_key
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     SignalClient.DH_RATCHET_PK_ID, 
                                                                     None))
        
        return decrypted_message.decode()
        
        
    def _initialize_state(self, phone_number: str, root_key):
        self._sessions[phone_number] = SignalClient.SessionState(True, 
                                                                 DiffieHellmanRatchet(), 
                                                                 KdfChain(root_key), 
                                                                 SymmetricRatchet(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                                                 SymmetricRatchet(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                                                 0)
        
        
    def _try_mimic(self, phone_number, session_phone_number, session):
        self._phone_number = phone_number
        self._sessions[session_phone_number] = session
        self._sessions[session_phone_number].dh_ratchet.generate_new_key_pair()
        
        trace('[SignalClient]', f'User now tries to be a user with phone {phone_number}')
        
        
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return KeyPair(private_key, public_key)
    
    
    @staticmethod
    def _dh(key_pair, public_key):
        return Curve.multiply_by_scalar(key_pair.private, public_key)
    
    
    @staticmethod
    def _produce_root_key(DH1, DH2, DH3, DH4):
        kdf_data = pickle.dumps(DH1) + pickle.dumps(DH2) + pickle.dumps(DH3) + pickle.dumps(DH4)
        return HKDF(kdf_data, KuznyechikMGM.KEY_SIZE, b'\x00' * 16, Streebog, 1)

----
## Демонстрация удачного выполнения протоколов

In [61]:
# Создаю сервер
srv = SignalServer()

In [62]:
# Пользователь Алиса
alice = SignalClient('+7 921 059-87-72')

[TRACE] [SignalClient] Identity key: KeyPair(private=2977995324410626473745930776702814269943676308620165048166008804907125303015133194240611156920704897500817482617546068649950153046304205911948917823713056, public=(10564690576545524781650009354492238065865291045771891662020006124390274856059604279754515394142369214689630195179684563483197463796879360511373537664206579, 3168617702184262197262540940150611485228724406933308956790226122708361907628607847537021173617734194668391382968693256461385913653940899157189407506452181))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=6944886150837330252966405809749343706448371280481882819425193422052935148128301179544732094398178228454834050348942913575542668528500704779412997802448770, public=(10283615243693098353102293788569493752731130684645333322196471372188375224646392795317680228506755102084618132623171276913352858569157576095608084774499252, 2304072525876319444718800401436580727182978376954597195762078562705009638515899517033833622290

In [63]:
# Пользователь Боб
bob = SignalClient('+7 991 174-51-34')

[TRACE] [SignalClient] Identity key: KeyPair(private=4811585871326956451911334427001391870769192533518608747949600512455786777486023567572333866024304883794925602182773732645080011464215456737078546503745356, public=(12722565611234258824958456668616044989648815106023450614995460634420256506088727771223278811060839042210624710671833474882466284262000113131892887172210867, 10630521879160299443003770209172114585179714912648245077464064851901161828763984514793109120320609078399293060648001248044828324428891201132975320640007889))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=9019893353832194620549762282949495819753953484322532635459889285604986863822418971119677682362550318690141215058823510701361761817177933617660882715007837, public=(9293490060927392177561129265374889072967857059512767295406918474177899997350922242891167141564603210588090407579721000842737088159902932816567016812658960, 5385402355476751466320356313030225253066862838398568816798315248503323044126429704024196868773

In [64]:
# Регистрирую всех 
srv.register(alice.phone_number, alice.public_keys)
srv.register(bob.phone_number, bob.public_keys)

[TRACE] [SignalServer] Registered client with phone number +7 921 059-87-72
[TRACE] [SignalServer] Registered client with phone number +7 991 174-51-34


In [65]:
# Алиса пытается пообщаться с Бобом
alice.send_message(bob.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 0
[TRACE] [SignalClient] Use 80049588000000000000008a41f386022cea8e26325cb3858beebf48d99e253d667e41ae3558d0b083cd4e5262b788219c305f44e1ef30ba0998bf73d46a52a9329bd0542a2dda7476c121b7c9008a40d54e8c898808a99a0de5220ce1c692b4ce6e2bf238365b74d2a9665f7ef38b06701b92d96a7ba612b16ee408a9165dac81bdd9291da09ae45813d1dc75e17f3c86942e80049589000000000000008a41b3e07059ef4a2556954fb1776df4ca16157538c3b06d87a2218c4ac345f86397dbf41f9f5a5df9007139ac6058522b77b54f524ab05c31fb97118104f69aeaf2008a41d12e088fd3ce1051b16ba92eca350a3925685fb6c7cee15d892566b8a5ed4244219e5468f11f2316eeb91735384da635634d66d3bca741d1fc04fd8e97e8f8ca0086942e as associated data
[TRACE] [SignalClient] Use bada9908d02f30c93e35839517afe62754e18eea2ec10387e3a0b94324088379 as root key
[TRACE] [SignalServer] Saved message with id X3DH_MSG from +7 921 059-87-72 to +7 991 174-51-34


In [66]:
# Боб полчает сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049588000000000000008a41f386022cea8e26325cb3858beebf48d99e253d667e41ae3558d0b083cd4e5262b788219c305f44e1ef30ba0998bf73d46a52a9329bd0542a2dda7476c121b7c9008a40d54e8c898808a99a0de5220ce1c692b4ce6e2bf238365b74d2a9665f7ef38b06701b92d96a7ba612b16ee408a9165dac81bdd9291da09ae45813d1dc75e17f3c86942e80049589000000000000008a41b3e07059ef4a2556954fb1776df4ca16157538c3b06d87a2218c4ac345f86397dbf41f9f5a5df9007139ac6058522b77b54f524ab05c31fb97118104f69aeaf2008a41d12e088fd3ce1051b16ba92eca350a3925685fb6c7cee15d892566b8a5ed4244219e5468f11f2316eeb91735384da635634d66d3bca741d1fc04fd8e97e8f8ca0086942e as associated data
[TRACE] [SignalClient] Use bada9908d02f30c93e35839517afe62754e18eea2ec10387e3a0b94324088379 as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 991 174-51-34 to +7 921 059-87-72


'Hi!'

In [67]:
# Это небольшой костыль, тут я получаю инициализирующее 
# сообщение с открытым ключом Боба
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id DH_RATCHET_PK from +7 991 174-51-34


In [68]:
# Алиса отсылает сообщение
alice.send_message(bob.phone_number, "What's up?", srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use ace7f7a671a2e74ed53661a8d4410bd2bba057ebd85028f581c878574b5adfec as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 921 059-87-72 to +7 991 174-51-34


In [69]:
# Боб принимает и расшифровывает
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use ace7f7a671a2e74ed53661a8d4410bd2bba057ebd85028f581c878574b5adfec as decryption key


"What's up?"

In [70]:
# Можно послать несколько сообщений подряд
bob.send_message(alice.phone_number, 'Some dummy phrase just to continue the dialog', srv)
bob.send_message(alice.phone_number, 'And another one', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 20feec243074bd9e32f8afe53c5caebaccbbc409176f7f21249f36413892551f as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 991 174-51-34 to +7 921 059-87-72
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 51f16b4eac4aa48479226547b8a7a866b4d9b9eee71068ee8c566619b72b86ea as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 991 174-51-34 to +7 921 059-87-72


In [71]:
# И получить оба. Одно...
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 20feec243074bd9e32f8afe53c5caebaccbbc409176f7f21249f36413892551f as decryption key


'Some dummy phrase just to continue the dialog'

In [72]:
# ... за другим
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 991 174-51-34
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 51f16b4eac4aa48479226547b8a7a866b4d9b9eee71068ee8c566619b72b86ea as decryption key


'And another one'

In [73]:
# Можно снова сменить направление отправки и 
# посмотреть, что произошел шаг DH-храповика
alice.send_message(bob.phone_number, 'Yet another DH ratchet step demo', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 9c47ee3008538a587679996683b7973c6a57270cca8c9dfaf1969e65fb670893 as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 921 059-87-72 to +7 991 174-51-34


In [74]:
# А на другой строне надо получить сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 9c47ee3008538a587679996683b7973c6a57270cca8c9dfaf1969e65fb670893 as decryption key


'Yet another DH ratchet step demo'

In [75]:
# Можно третьего пользователя создать
carol = SignalClient('+7 970 505-44-25')
srv.register(carol.phone_number, carol.public_keys)

[TRACE] [SignalClient] Identity key: KeyPair(private=5763849460220260924609685745041929536448933870728836738794202779400406286435320084756380042010011821080093930227205644274954917902701536675879020040295666, public=(1274562867337817148243162014257321292842715823815368912009656764818611717184862715240896685844637759215284123352003560716572407238625121069654489362482350, 5890144673997187807965427108545775554986309174577904263322671449449395097740490966833680750934512514725990461134595481791847710506187761521432134830465017))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=6518671968298517561693707519182925110320874926306175084694715993266156421009060573936908772075056332812569458250956215183658201014146738050516519777173257, public=(5515513551216091752011679272067541588708171711694666705800919035883983547583645542608182182294333967336024496243919702412901521307987200306706437665934770, 124800987063352545729417083554988626698465301374958885721868165773381893915810394708794355469113

In [76]:
alice.send_message(bob.phone_number, "I'm making new friends", srv)
alice.send_message(carol.phone_number, 'Hi, Carol!', srv)

[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use c669a004e4bb7236f4678c488329bf919cba757415eb619fa6e79842518aeaea as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 921 059-87-72 to +7 991 174-51-34
[TRACE] [SignalServer] Using OPK at 9
[TRACE] [SignalClient] Use 80049588000000000000008a41f386022cea8e26325cb3858beebf48d99e253d667e41ae3558d0b083cd4e5262b788219c305f44e1ef30ba0998bf73d46a52a9329bd0542a2dda7476c121b7c9008a40d54e8c898808a99a0de5220ce1c692b4ce6e2bf238365b74d2a9665f7ef38b06701b92d96a7ba612b16ee408a9165dac81bdd9291da09ae45813d1dc75e17f3c86942e80049587000000000000008a40ae445ac00105e882c7e97db2ce62e9441792b1433ad2619d07456d011184598db4cc47011d29b71748aea5d74ee8314547d618bc1b46b0013ae26e45fbee55188a40f91f2f347518cdd63f8bf5a73cf28ffb69d277a2919485688cac396c33ae7bd7957eb48402ea44eeef475ed053b6719f96b53ed317d2e0a38fa57ecc6c6d767086942e as associated data
[TRACE] [SignalClient] Use ed44829ade97d9f4d9332bda531445618d6e8eadb93aaa8f34a66c822b6

In [77]:
carol.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049588000000000000008a41f386022cea8e26325cb3858beebf48d99e253d667e41ae3558d0b083cd4e5262b788219c305f44e1ef30ba0998bf73d46a52a9329bd0542a2dda7476c121b7c9008a40d54e8c898808a99a0de5220ce1c692b4ce6e2bf238365b74d2a9665f7ef38b06701b92d96a7ba612b16ee408a9165dac81bdd9291da09ae45813d1dc75e17f3c86942e80049587000000000000008a40ae445ac00105e882c7e97db2ce62e9441792b1433ad2619d07456d011184598db4cc47011d29b71748aea5d74ee8314547d618bc1b46b0013ae26e45fbee55188a40f91f2f347518cdd63f8bf5a73cf28ffb69d277a2919485688cac396c33ae7bd7957eb48402ea44eeef475ed053b6719f96b53ed317d2e0a38fa57ecc6c6d767086942e as associated data
[TRACE] [SignalClient] Use ed44829ade97d9f4d9332bda531445618d6e8eadb93aaa8f34a66c822b64f19d as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 970 505-44-25 to +7 921 059-87-72


'Hi, Carol!'

In [78]:
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 921 059-87-72
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use c669a004e4bb7236f4678c488329bf919cba757415eb619fa6e79842518aeaea as decryption key


"I'm making new friends"

----
## Демонстрация неуспешного выполнения протоколов

In [79]:
# Пользователь, который как-то украл подпись и подписанный ключ
# Но его идентификационный ключ не позволяет проверить подпись
mallory = SignalClient('+1 202-918-2132', fake_ik=True)

[TRACE] [SignalClient] Identity key: KeyPair(private=5240225487891293689066449601046501533674999614581136523473886658543701076836430664038081798933095742580802363432536191351928560225939404564735926008093998, public=(8052025988214663741839754132888807622695853716757117504389637038922223751945014233480466006149103941696370016676478441046870901076927775180390118218078383, 3665754808622162401365402882095595489626452050862686099822497205130437015169213810296525062414839904633274523545902489921776007429991948615253162425209188))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=7788518855717057382633701660449043971157833138778720176155306208194658099791774524797733104362699660757507363690844407238882909339121135654070672720099870, public=(3329005016708804426928589316177914767372104238048375366707051745505360472015101276599055468284521038633885899324722712941748921782388256596974897801559618, 156171008166655338077046127330375543043836809795336735199766043579153359388818284309600167777230

In [80]:
# Надо зарегистрироваться
srv.register(mallory.phone_number, mallory.public_keys)

[TRACE] [SignalServer] Registered client with phone number +1 202-918-2132


In [81]:
# Алиса получает ключи с сервера и обнаруживает несоответствие
alice.send_message(mallory.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 9
[TRACE] [SignalClient] Cannot verify signature of client with phone +1 202-918-2132


In [82]:
# Еще один пользователь-враг -- Ева
eve = SignalClient('+1 539-567-2473')

[TRACE] [SignalClient] Identity key: KeyPair(private=12834273399463691714071951748892377708661184816701126516493816717477518633748362714379891519375658542259317829204294594731787292807404731569305128661358559, public=(12998232523913368600519700023930146316177651305796351165603518516545837303408889066820218077414405543497327437841799642674475554507168013075856882557580220, 3781820757285636120131407392287251168132999315292286709017364426704896805981366163036995181666962745032743860479840540081968523800580349190579592208390423))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=2849152170942888984645832563303343743186811059323753269042976954002649891161804470669815378938136948041533545763405694449302892084948010166694052347844337, public=(10648055206094319318754291730817566666130856373012495660271030513252914536493708467115047910237226547512623595999882966476614268223416643963170703427615067, 510799089800197988327788898267959725838869834691291789670896200590769927757203481927931326246

In [83]:
# Она пытается прикинуться Алисой в разговоре с Бобом
eve._try_mimic(alice.phone_number, bob.phone_number, 
               alice._sessions[bob.phone_number])

[TRACE] [SignalClient] User now tries to be a user with phone +7 921 059-87-72


In [84]:
# Боб отправляет Алисе соообщение
bob.send_message(alice.phone_number, 'Hi!', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 5166495bbaf8566e9893fb175a5c33872993d07f47baa08d2bb0bac9308a97f3 as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 991 174-51-34 to +7 921 059-87-72


In [85]:
# А вот получить его пытается Ева, но расшифровать не получится
eve.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 52fa54f98b8cc9f35e5a00f6d293aa612f9b88efaa5747cc0570b4e6efd8b798 as decryption key
[TRACE] [SignalClient] Cannot decrypt message
